In [1]:
import pandas as pd
import numpy as np
import os  
import sys  
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import LatentDirichletAllocation
import time

In [2]:
By_sentence_pos=pd.read_csv('By_sentence_pos.csv')
By_sentence_pos['pos']=By_sentence_pos['pos'].apply(eval)

In [3]:
##過濾掉特別磁性
ckip_pos=['VA','VB','VC','VD','VE','VH','VHC','VK','VJ','VL','VCL','Na','Nb','Nc','Nd','Nv','FW']
By_sentence_pos['token']=By_sentence_pos['pos'].apply(lambda x:[i[0] for i in x if (i[1] in ckip_pos)] )
from collections import Counter
By_sentence_pos['Counter']=By_sentence_pos['token'].apply(Counter)

In [ ]:
classiff_ckip=pd.read_csv('classiff_ckip.csv')

class_token=[]
for i in classiff_ckip.values:
    for k in i:
        if k!='nan':
            class_token.append(k)
            
classif=classiff_ckip.to_dict()
service_list=np.array(list(classif['service'].values()))
classif['service']=service_list[service_list!='nan']
place_list=np.array(list(classif['place'].values()))
classif['place']=place_list[place_list!='nan']
clean_list=np.array(list(classif['clean'].values()))
classif['clean']=clean_list[clean_list!='nan']
cp_list=np.array(list(classif['cp'].values()))
classif['cp']=cp_list[cp_list!='nan']
comfort_list=np.array(list(classif['comfort'].values()))
classif['comfort']=comfort_list[comfort_list!='nan']
sleep_list=np.array(list(classif['sleep'].values()))
classif['sleep']=sleep_list[sleep_list!='nan']

In [ ]:
def find_Category(x):
    cate=[]
    token_set=set(x)
    for category in classif.keys():
        cate_set=set(classif[category])
        if cate_set&token_set!=set():
            cate.append(category)
    if cate==[]:
        cate=None
    return cate
By_sentence_pos['category']=By_sentence_pos['token'].apply(find_Category)

In [ ]:
no_topic=By_sentence_pos[By_sentence_pos.category.isnull()]

In [ ]:
no_topic['no_class_token']=no_topic['token'].apply(lambda x:[i for i in x if i not in class_token ])
no_topic['none_Class']=no_topic['no_class_token'].apply(lambda x:' '.join(x))

In [ ]:
# 文章字詞矩陣，沒有加權 ****注意單辭被排除在外
vectorizer = CountVectorizer(min_df = 1,ngram_range= (0,1))  
count = vectorizer.fit_transform(no_topic['none_Class']) 
transformer=TfidfTransformer()  

In [ ]:
def list_top_words(model, feature_names, n_top_words,j):
    #打印每个主题下权重较高的term
    ans=[]
    for i in range(j):
        word= list([feature_names[k] for k in model.components_[i].argsort()[:-n_top_words - 1:-1]])        
        pro=list(proba[i][k] for k in model.components_[i].argsort()[:-n_top_words - 1:-1])
        ans.append([word,pro])
    return(ans)

    #打印主题-词语分布矩阵
    #print (model.components_)
tf_feature_names = vectorizer.get_feature_names()
all_result=[]
perplexity=[]
n_top_words=50
#tf_feature_names = vectorizer.get_feature_names()
for j in range(3,11):
    tstart = time.time()
    print(j)
    lda = LatentDirichletAllocation(j, 
                                max_iter=3000,                #看要不要改大一點
                                learning_method='batch',n_jobs=10)
    lda.fit(count) #tf即为Document_word Sparse Matrix 
    proba=lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    result=list_top_words(lda, tf_feature_names, n_top_words,j)
    perple=lda.perplexity(count)
    all_result.append(result)
    perplexity.append(perple)
    tend = time.time()#計時結束
    print ("It cost %f sec" % (tend - tstart))#會自動做近位

In [ ]:
#topic存成excel
writer = pd.ExcelWriter('LDA_newtopic.xlsx')#, engine='xlsxwriter')
for num_ , result in enumerate(all_result):
    topic_df=pd.DataFrame()
    number=num_+2
    for index_,topics in enumerate(result):
        topic_df[f'topic_{index_}_word']=topics[0]
        topic_df[f'topic_{index_}_prob']=topics[1]
    topic_df.to_excel(writer,sheet_name=f'{number}_topic')
writer.save()

#### 繪圖-perplexity
* 根據該結果找出合適主題數

In [ ]:
import matplotlib.pyplot as plt
def graph_draw(topic,perplexity):
    x = topic
    y = perplexity
    plt.plot(x,y,color = 'red',linewidth=2)
    plt.xlabel(' Number of Topic')
    plt.ylabel('Perplexity')
    plt.show()

In [ ]:
topic = [2,3,4,5,6,7,8,9,10]
graph_draw(topic,perplexity)